In [107]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [108]:
dataframes = pandas.read_csv("../data/files/test_set.csv", delimiter=',',error_bad_lines=False,compression='gzip')
dataframes.head(3)


,description,floor,lat-lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,...,amenities,patio,parrilla,pileta,quincho,rooms1,rooms2,surface_total_in_m21,surface_total_in_m22,id
0,0.787823,NaN,0.104345,0.578584,0.718093,1.265273,0.333333,0.378310,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.333333,0.5,1.0,0.0,3632
1,0.125275,NaN,NaN,0.624856,0.779815,1.265273,NaN,0.671307,NaN,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,3633
2,0.582238,NaN,0.535028,0.727655,0.563185,1.265273,0.166667,0.378310,0.000014,0.000265,...,0.0,0.0,1.0,0.0,0.0,0.333333,0.5,0.0,0.0,2263404


In [109]:
columns  = ['place_name','place_with_parent_names','property_type','surface_covered_in_m2','surface_total_in_m2','chalet','esquina','cochera','aire acondicionado','living','a estrenar','transporte','cocina','amenities','patio','parrilla','pileta']
test = dataframes.reindex(columns=columns)
test.fillna(test.mean(),inplace= True)
test.head(10)

,place_name,place_with_parent_names,property_type,surface_covered_in_m2,surface_total_in_m2,chalet,esquina,cochera,aire acondicionado,living,a estrenar,transporte,cocina,amenities,patio,parrilla,pileta
0,0.578584,0.718093,1.265273,0.000110,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.624856,0.779815,1.265273,0.000110,0.000000,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.727655,0.563185,1.265273,0.000014,0.000265,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
3,0.706435,0.223811,1.265273,0.000012,0.000195,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,0.706435,0.223811,1.265273,0.000015,0.000255,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
5,0.706435,0.223811,1.265273,0.000012,0.000195,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
6,0.706435,0.223811,1.265273,0.000012,0.000195,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
7,0.706435,0.223811,1.265273,0.000012,0.000195,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
8,0.972850,0.550633,1.265273,0.000020,0.000335,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
9,0.460782,0.743727,1.265273,0.000014,0.000235,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0


In [110]:
xtest = test.values
 
xtest=xtest[:,0:17]

In [111]:
xtest 

array([[ 0.57858404,  0.71809251,  1.26527256, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.6248561 ,  0.77981516,  1.26527256, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.7276547 ,  0.56318484,  1.26527256, ...,  0.        ,
         1.        ,  0.        ],
       ..., 
       [ 0.01032999,  0.33224179, -0.07793853, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.66296043,  0.93310726,  1.26527256, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.35741933,  0.92692605, -0.07793853, ...,  0.        ,
         1.        ,  1.        ]])

In [112]:
dataframe = pandas.read_csv("../data/files/training_set.csv", delimiter=',',error_bad_lines=False,compression='gzip')
dataframe.head(5)

,id,description,floor,lat-lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,...,cocina,amenities,patio,parrilla,pileta,quincho,rooms1,rooms2,surface_total_in_m21,surface_total_in_m22
0,0,0.758721,0.070631,0.526570,0.287817,0.632960,0.0,0.268115,0.000000,0.000055,...,1.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.0,0.0,0.0
1,1,0.606186,0.070631,0.511955,0.879101,0.444374,0.0,0.666667,0.000000,0.000040,...,1.0,0.0,0.0,0.0,1.0,1.0,0.666667,1.0,1.0,0.0
2,2,0.707316,0.070631,0.353935,0.422263,0.923406,0.0,0.268115,0.788018,0.000049,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0
3,3,0.665816,0.070631,0.653331,0.863868,0.024805,1.0,0.000000,0.378310,0.000011,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,4,0.377035,0.070631,0.443158,0.297724,1.000000,0.0,0.268115,0.000000,0.000108,...,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [113]:
columns = ['place_name','place_with_parent_names','property_type','surface_covered_in_m2','surface_total_in_m2','chalet','esquina','cochera','aire acondicionado','living','a estrenar','transporte','cocina','amenities','patio','parrilla','pileta','price_aprox_usd']
entrenamiento=dataframe.reindex(columns=columns)
entrenamiento.fillna(entrenamiento.mean(),inplace= True)
entrenamiento.head(5)

,place_name,place_with_parent_names,property_type,surface_covered_in_m2,surface_total_in_m2,chalet,esquina,cochera,aire acondicionado,living,a estrenar,transporte,cocina,amenities,patio,parrilla,pileta,price_aprox_usd
0,0.287817,0.632960,0.0,0.000055,0.001150,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,178000.0
1,0.879101,0.444374,0.0,0.000040,0.000650,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,230000.0
2,0.422263,0.923406,0.0,0.000049,0.000910,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,240000.0
3,0.863868,0.024805,1.0,0.000011,0.000015,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,77000.0
4,0.297724,1.000000,0.0,0.000108,0.006040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,470000.0


In [114]:
dataset = entrenamiento.values
 
X = dataset[0:,0:17]
Y = dataset[0:,17:18]

In [115]:
X[:4]


array([[  2.87817478e-01,   6.32959603e-01,   0.00000000e+00,
          5.51917048e-05,   1.15000000e-03,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00],
       [  8.79101496e-01,   4.44373754e-01,   0.00000000e+00,
          3.97750275e-05,   6.50000000e-04,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00],
       [  4.22262777e-01,   9.23406028e-01,   0.00000000e+00,
          4.91575028e-05,   9.10000000e-04,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00

In [116]:
Y[:4]

array([[ 178000.],
       [ 230000.],
       [ 240000.],
       [  77000.]])

In [117]:
 
# define base model
def baseline_model():

    # create model
    model = Sequential()
    model.add(Dense(17, input_dim=17, init="normal", activation='relu'))#cantidad de entradas es el 4,inputdim
    model.add(Dense(8, init='normal', activation='relu'))
    model.add(Dense(1, init="normal"))#salida
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    
    return model

In [118]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=50, batch_size=5, verbose=2)#epocas cantidad de ciblos 100 tomaban 

    # round predictions

In [119]:
kfold = KFold(n_splits=5, random_state=seed) #splits 5 o 10 
results = cross_val_score(estimator, X, Y, cv=kfold)

print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
estimator.fit(X, Y)
prediction = estimator.predict(xtest)
 
# round predictionsrounded =
print(prediction) 

Epoch 1/2
81s - loss: 142588555564.3618
Epoch 2/2
89s - loss: 128443999871.9308
Epoch 1/2
79s - loss: 159007597735.2354
Epoch 2/2
76s - loss: 145262041135.4863
Results: 135563094910.19 (9278545016.76) MSE
Epoch 1/2
173s - loss: 144533396121.1622
Epoch 2/2
158s - loss: 134146548670.5231
[ 226949.25      411626.78125   155736.15625  ...,  241028.171875
  279769.21875   552712.8125  ]


In [120]:
for x in prediction:
     print(x)
  

226949.0
411627.0
155736.0
170682.0
222618.0
170682.0
170682.0
170682.0
319589.0
237500.0
237500.0
237500.0
287526.0
326802.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
215814.0
215815.0
215815.0
215815.0
215815.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
163879.0
135694.0
397286.0
287526.0
287526.0
236173.0
174940.0
185398.0
130084.0
206425.0
166848.0
215454.0
204633.0
363128.0
172821.0
139178.0
130393.0
171161.0
296437.0
129414.0
494252.0
141730.0
292656.0
261229.0
216482.0
157340.0
167106.0
330750.0
167106.0
210448.0
136131.0
250012.0
222578.0
125297.0
136131.0
136131.0
136131.0
136131.0
390530.0
136131.0
136150.0
390530.0
136163.0
127971.0
171230.0
222944.0
312368.0
389250.0
99509.6
252263.0
27819.5
241303.0
145438.0
147898.0
147898.0
377761.0
173496.0
215458.0
131941.0
468904.0
345638.0
208945.0
285488.0
285488.0
418763.0
94872.1
93078.5
276418.0
230871.0
183822.0
108927.0
23885

In [130]:
precios = pandas.DataFrame( dataframes['id'],columns={'id'})
precios.head(5)

,id
0,3632
1,3633
2,2263404
3,2263405
4,2263406


In [131]:


precios['price_usd']=prediction

In [132]:
precios.head(4)


,id,price_usd
0,3632,226949.25000
1,3633,411626.78125
2,2263404,155736.15625
3,2263405,170682.40625


In [133]:
precios.to_csv("../data/files/resultados.csv",index=False)